In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!zip -F file.zip --out file-large.zip
!unzip file-large.zip
!unzip "/content/drive/My Drive/img_align_celeba.zip"
!cat '/content/drive/My Drive/utils.py'
!cat '/content/drive/My Drive/ops.py'
!cat '/content/drive/My Drive/models_64x64.py'
import sys
sys.path.append('/content/drive/My Drive')

Streaming output truncated to the last 5000 lines.
 extracting: img_align_celeba/198145.jpg  
 extracting: img_align_celeba/198146.jpg  
 extracting: img_align_celeba/198147.jpg  
 extracting: img_align_celeba/198148.jpg  
 extracting: img_align_celeba/198149.jpg  
 extracting: img_align_celeba/198150.jpg  
 extracting: img_align_celeba/198151.jpg  
 extracting: img_align_celeba/198152.jpg  
 extracting: img_align_celeba/198153.jpg  
 extracting: img_align_celeba/198154.jpg  
 extracting: img_align_celeba/198155.jpg  
 extracting: img_align_celeba/198156.jpg  
 extracting: img_align_celeba/198157.jpg  
 extracting: img_align_celeba/198158.jpg  
 extracting: img_align_celeba/198159.jpg  
 extracting: img_align_celeba/198160.jpg  
 extracting: img_align_celeba/198161.jpg  
 extracting: img_align_celeba/198162.jpg  
 extracting: img_align_celeba/198163.jpg  
 extracting: img_align_celeba/198164.jpg  
 extracting: img_align_celeba/198165.jpg  
 extracting: img_align_celeba/198166.jpg  
 ex

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import glob
import utils
import traceback
import numpy as np
import tensorflow as tf
import models_64x64 as models


""" param """
epoch = 50
batch_size = 64
lr = 0.0002
z_dim = 100
clip = 0.01
n_critic = 5
gpu_id = 3

''' data '''
# you should prepare your own data in ./data/img_align_celeba
# celeba original size is [218, 178, 3]


def preprocess_fn(img):
    crop_size = 108
    re_size = 64
    img = tf.image.crop_to_bounding_box(img, (218 - crop_size) // 2, (178 - crop_size) // 2, crop_size, crop_size)
    img = tf.to_float(tf.image.resize_images(img, [re_size, re_size], method=tf.image.ResizeMethod.BICUBIC)) / 127.5 - 1
    return img

img_paths = glob.glob('/content/img_align_celeba/')
data_pool = utils.DiskImageData('/content/img_align_celeba/', batch_size, shape=[218, 178, 3], preprocess_fn=preprocess_fn)

tf.reset_default_graph()


""" graphs """
with tf.device('/gpu:%d' % gpu_id):
    ''' models '''
    generator = models.generator
    discriminator = models.discriminator

    ''' graph '''
    # inputs
    real = tf.placeholder(tf.float32, shape=[None, 64, 64, 3])
    z = tf.placeholder(tf.float32, shape=[None, z_dim])

    # generate
    fake = generator(z, reuse=False)

    # dicriminate
    r_logit = discriminator(real, reuse=False)
    f_logit = discriminator(fake)

    # losses
    wd = tf.reduce_mean(r_logit) - tf.reduce_mean(f_logit)
    d_loss = -wd
    g_loss = -tf.reduce_mean(f_logit)

    # otpims
    d_var = utils.trainable_variables('discriminator')
    g_var = utils.trainable_variables('generator')
    d_step_ = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(d_loss, var_list=d_var)
    with tf.control_dependencies([d_step_]):
        d_step = tf.group(*(tf.assign(var, tf.clip_by_value(var, -clip, clip)) for var in d_var))
    g_step = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(g_loss, var_list=g_var)

    # summaries
    d_summary = utils.summary({wd: 'wd'})
    g_summary = utils.summary({g_loss: 'g_loss'})

    # sample
    f_sample = generator(z, training=False)


""" train """
''' init '''
# session
sess = utils.session()
# iteration counter
it_cnt, update_cnt = utils.counter()
# saver
saver = tf.train.Saver(max_to_keep=5)
# summary writer
summary_writer = tf.summary.FileWriter('/content/drive/My Drive/GP/summaries/celeba_wgan', sess.graph)

''' initialization '''
ckpt_dir = '/content/drive/My Drive/GP/checkpoints/celeba_wgan'
utils.mkdir(ckpt_dir + '/')
if not utils.load_checkpoint(ckpt_dir, sess):
    sess.run(tf.global_variables_initializer())

''' train '''
try:
    z_ipt_sample = np.random.normal(size=[100, z_dim])

    batch_epoch = len(data_pool) // (batch_size * n_critic)
    max_it = epoch * batch_epoch
    for it in range(sess.run(it_cnt), max_it):
        sess.run(update_cnt)

        # which epoch
        epoch = it // batch_epoch
        it_epoch = it % batch_epoch + 1

        # train D
        if it < 25:
            c_iter = 100
        else:
            c_iter = n_critic
        for i in range(n_critic):
            # batch data
            real_ipt = data_pool.batch()
            z_ipt = np.random.normal(size=[batch_size, z_dim])
            d_summary_opt, _ = sess.run([d_summary, d_step], feed_dict={real: real_ipt, z: z_ipt})
        summary_writer.add_summary(d_summary_opt, it)

        # train G
        z_ipt = np.random.normal(size=[batch_size, z_dim])
        g_summary_opt, _ = sess.run([g_summary, g_step], feed_dict={z: z_ipt})
        summary_writer.add_summary(g_summary_opt, it)

        # display
        if it % 1 == 0:
            print("Epoch: (%3d) (%5d/%5d)" % (epoch, it_epoch, batch_epoch))

        # save
        if (it + 1) % 1000 == 0:
            save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_epoch, batch_epoch))
            print('Model saved in file: % s' % save_path)

        # sample
        if (it + 1) % 100 == 0:
            f_sample_opt = sess.run(f_sample, feed_dict={z: z_ipt_sample})

            save_dir = '/content/drive/My Drive/GP/sample_images_while_training/celeba_wgan'
            utils.mkdir(save_dir + '/')
            utils.imwrite(utils.immerge(f_sample_opt, 10, 10), '%s/Epoch_(%d)_(%dof%d).jpg' % (save_dir, epoch, it_epoch, batch_epoch))

except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

#Data is  202599
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuf

Epoch: (  0) (  101/  633)
Epoch: (  0) (  102/  633)
Epoch: (  0) (  103/  633)
Epoch: (  0) (  104/  633)
Epoch: (  0) (  105/  633)
Epoch: (  0) (  106/  633)
Epoch: (  0) (  107/  633)
Epoch: (  0) (  108/  633)
Epoch: (  0) (  109/  633)
Epoch: (  0) (  110/  633)
Epoch: (  0) (  111/  633)
Epoch: (  0) (  112/  633)
Epoch: (  0) (  113/  633)
Epoch: (  0) (  114/  633)
Epoch: (  0) (  115/  633)
Epoch: (  0) (  116/  633)
Epoch: (  0) (  117/  633)
Epoch: (  0) (  118/  633)
Epoch: (  0) (  119/  633)
Epoch: (  0) (  120/  633)
Epoch: (  0) (  121/  633)
Epoch: (  0) (  122/  633)
Epoch: (  0) (  123/  633)
Epoch: (  0) (  124/  633)
Epoch: (  0) (  125/  633)
Epoch: (  0) (  126/  633)
Epoch: (  0) (  127/  633)
Epoch: (  0) (  128/  633)
Epoch: (  0) (  129/  633)
Epoch: (  0) (  130/  633)
Epoch: (  0) (  131/  633)
Epoch: (  0) (  132/  633)
Epoch: (  0) (  133/  633)
Epoch: (  0) (  134/  633)
Epoch: (  0) (  135/  633)
Epoch: (  0) (  136/  633)
Epoch: (  0) (  137/  633)
E

Epoch: (  0) (  200/  633)
Epoch: (  0) (  201/  633)
Epoch: (  0) (  202/  633)
Epoch: (  0) (  203/  633)
Epoch: (  0) (  204/  633)
Epoch: (  0) (  205/  633)
Epoch: (  0) (  206/  633)
Epoch: (  0) (  207/  633)
Epoch: (  0) (  208/  633)
Epoch: (  0) (  209/  633)
Epoch: (  0) (  210/  633)
Epoch: (  0) (  211/  633)
Epoch: (  0) (  212/  633)
Epoch: (  0) (  213/  633)
Epoch: (  0) (  214/  633)
Epoch: (  0) (  215/  633)
Epoch: (  0) (  216/  633)
Epoch: (  0) (  217/  633)
Epoch: (  0) (  218/  633)
Epoch: (  0) (  219/  633)
Epoch: (  0) (  220/  633)
Epoch: (  0) (  221/  633)
Epoch: (  0) (  222/  633)
Epoch: (  0) (  223/  633)
Epoch: (  0) (  224/  633)
Epoch: (  0) (  225/  633)
Epoch: (  0) (  226/  633)
Epoch: (  0) (  227/  633)
Epoch: (  0) (  228/  633)
Epoch: (  0) (  229/  633)
Epoch: (  0) (  230/  633)
Epoch: (  0) (  231/  633)
Epoch: (  0) (  232/  633)
Epoch: (  0) (  233/  633)
Epoch: (  0) (  234/  633)
Epoch: (  0) (  235/  633)
Epoch: (  0) (  236/  633)
E

Epoch: (  0) (  300/  633)
Epoch: (  0) (  301/  633)
Epoch: (  0) (  302/  633)
Epoch: (  0) (  303/  633)
Epoch: (  0) (  304/  633)
Epoch: (  0) (  305/  633)
Epoch: (  0) (  306/  633)
Epoch: (  0) (  307/  633)
Epoch: (  0) (  308/  633)
Epoch: (  0) (  309/  633)
Epoch: (  0) (  310/  633)
Epoch: (  0) (  311/  633)
Epoch: (  0) (  312/  633)
Epoch: (  0) (  313/  633)
Epoch: (  0) (  314/  633)
Epoch: (  0) (  315/  633)
Epoch: (  0) (  316/  633)
Epoch: (  0) (  317/  633)
Epoch: (  0) (  318/  633)
Epoch: (  0) (  319/  633)
Epoch: (  0) (  320/  633)
Epoch: (  0) (  321/  633)
Epoch: (  0) (  322/  633)
Epoch: (  0) (  323/  633)
Epoch: (  0) (  324/  633)
Epoch: (  0) (  325/  633)
Epoch: (  0) (  326/  633)
Epoch: (  0) (  327/  633)
Epoch: (  0) (  328/  633)
Epoch: (  0) (  329/  633)
Epoch: (  0) (  330/  633)
Epoch: (  0) (  331/  633)
Epoch: (  0) (  332/  633)
Epoch: (  0) (  333/  633)
Epoch: (  0) (  334/  633)
Epoch: (  0) (  335/  633)
Epoch: (  0) (  336/  633)
E

Epoch: (  0) (  400/  633)
Epoch: (  0) (  401/  633)
Epoch: (  0) (  402/  633)
Epoch: (  0) (  403/  633)
Epoch: (  0) (  404/  633)
Epoch: (  0) (  405/  633)
Epoch: (  0) (  406/  633)
Epoch: (  0) (  407/  633)
Epoch: (  0) (  408/  633)
Epoch: (  0) (  409/  633)
Epoch: (  0) (  410/  633)
Epoch: (  0) (  411/  633)
Epoch: (  0) (  412/  633)
Epoch: (  0) (  413/  633)
Epoch: (  0) (  414/  633)
Epoch: (  0) (  415/  633)
Epoch: (  0) (  416/  633)
Epoch: (  0) (  417/  633)
Epoch: (  0) (  418/  633)
Epoch: (  0) (  419/  633)
Epoch: (  0) (  420/  633)
Epoch: (  0) (  421/  633)
Epoch: (  0) (  422/  633)
Epoch: (  0) (  423/  633)
Epoch: (  0) (  424/  633)
Epoch: (  0) (  425/  633)
Epoch: (  0) (  426/  633)
Epoch: (  0) (  427/  633)
Epoch: (  0) (  428/  633)
Epoch: (  0) (  429/  633)
Epoch: (  0) (  430/  633)
Epoch: (  0) (  431/  633)
Epoch: (  0) (  432/  633)
Epoch: (  0) (  433/  633)
Epoch: (  0) (  434/  633)
Epoch: (  0) (  435/  633)
Epoch: (  0) (  436/  633)
E

...

Epoch: ( 49) (  183/  633)
Epoch: ( 49) (  184/  633)
Epoch: ( 49) (  185/  633)
Epoch: ( 49) (  186/  633)
Epoch: ( 49) (  187/  633)
Epoch: ( 49) (  188/  633)
Epoch: ( 49) (  189/  633)
Epoch: ( 49) (  190/  633)
Epoch: ( 49) (  191/  633)
Epoch: ( 49) (  192/  633)
Epoch: ( 49) (  193/  633)
Epoch: ( 49) (  194/  633)
Epoch: ( 49) (  195/  633)
Epoch: ( 49) (  196/  633)
Epoch: ( 49) (  197/  633)
Epoch: ( 49) (  198/  633)
Epoch: ( 49) (  199/  633)
Epoch: ( 49) (  200/  633)
Epoch: ( 49) (  201/  633)
Epoch: ( 49) (  202/  633)
Epoch: ( 49) (  203/  633)
Epoch: ( 49) (  204/  633)
Epoch: ( 49) (  205/  633)
Epoch: ( 49) (  206/  633)
Epoch: ( 49) (  207/  633)
Epoch: ( 49) (  208/  633)
Epoch: ( 49) (  209/  633)
Epoch: ( 49) (  210/  633)
Epoch: ( 49) (  211/  633)
Epoch: ( 49) (  212/  633)
Epoch: ( 49) (  213/  633)
Epoch: ( 49) (  214/  633)
Epoch: ( 49) (  215/  633)
Epoch: ( 49) (  216/  633)
Epoch: ( 49) (  217/  633)
Epoch: ( 49) (  218/  633)
Epoch: ( 49) (  219/  633)
E

Epoch: ( 49) (  283/  633)
Epoch: ( 49) (  284/  633)
Epoch: ( 49) (  285/  633)
Epoch: ( 49) (  286/  633)
Epoch: ( 49) (  287/  633)
Epoch: ( 49) (  288/  633)
Epoch: ( 49) (  289/  633)
Epoch: ( 49) (  290/  633)
Epoch: ( 49) (  291/  633)
Epoch: ( 49) (  292/  633)
Epoch: ( 49) (  293/  633)
Epoch: ( 49) (  294/  633)
Epoch: ( 49) (  295/  633)
Epoch: ( 49) (  296/  633)
Epoch: ( 49) (  297/  633)
Epoch: ( 49) (  298/  633)
Epoch: ( 49) (  299/  633)
Epoch: ( 49) (  300/  633)
Epoch: ( 49) (  301/  633)
Epoch: ( 49) (  302/  633)
Epoch: ( 49) (  303/  633)
Epoch: ( 49) (  304/  633)
Epoch: ( 49) (  305/  633)
Epoch: ( 49) (  306/  633)
Epoch: ( 49) (  307/  633)
Epoch: ( 49) (  308/  633)
Epoch: ( 49) (  309/  633)
Epoch: ( 49) (  310/  633)
Epoch: ( 49) (  311/  633)
Epoch: ( 49) (  312/  633)
Epoch: ( 49) (  313/  633)
Epoch: ( 49) (  314/  633)
Epoch: ( 49) (  315/  633)
Epoch: ( 49) (  316/  633)
Epoch: ( 49) (  317/  633)
Epoch: ( 49) (  318/  633)
Epoch: ( 49) (  319/  633)
E

Epoch: ( 49) (  383/  633)
Epoch: ( 49) (  384/  633)
Epoch: ( 49) (  385/  633)
Epoch: ( 49) (  386/  633)
Epoch: ( 49) (  387/  633)
Epoch: ( 49) (  388/  633)
Epoch: ( 49) (  389/  633)
Epoch: ( 49) (  390/  633)
Epoch: ( 49) (  391/  633)
Epoch: ( 49) (  392/  633)
Epoch: ( 49) (  393/  633)
Epoch: ( 49) (  394/  633)
Epoch: ( 49) (  395/  633)
Epoch: ( 49) (  396/  633)
Epoch: ( 49) (  397/  633)
Epoch: ( 49) (  398/  633)
Epoch: ( 49) (  399/  633)
Epoch: ( 49) (  400/  633)
Epoch: ( 49) (  401/  633)
Epoch: ( 49) (  402/  633)
Epoch: ( 49) (  403/  633)
Epoch: ( 49) (  404/  633)
Epoch: ( 49) (  405/  633)
Epoch: ( 49) (  406/  633)
Epoch: ( 49) (  407/  633)
Epoch: ( 49) (  408/  633)
Epoch: ( 49) (  409/  633)
Epoch: ( 49) (  410/  633)
Epoch: ( 49) (  411/  633)
Epoch: ( 49) (  412/  633)
Epoch: ( 49) (  413/  633)
Epoch: ( 49) (  414/  633)
Epoch: ( 49) (  415/  633)
Epoch: ( 49) (  416/  633)
Epoch: ( 49) (  417/  633)
Epoch: ( 49) (  418/  633)
Epoch: ( 49) (  419/  633)
E

Epoch: ( 49) (  483/  633)
Epoch: ( 49) (  484/  633)
Epoch: ( 49) (  485/  633)
Epoch: ( 49) (  486/  633)
Epoch: ( 49) (  487/  633)
Epoch: ( 49) (  488/  633)
Epoch: ( 49) (  489/  633)
Epoch: ( 49) (  490/  633)
Epoch: ( 49) (  491/  633)
Epoch: ( 49) (  492/  633)
Epoch: ( 49) (  493/  633)
Epoch: ( 49) (  494/  633)
Epoch: ( 49) (  495/  633)
Epoch: ( 49) (  496/  633)
Epoch: ( 49) (  497/  633)
Epoch: ( 49) (  498/  633)
Epoch: ( 49) (  499/  633)
Epoch: ( 49) (  500/  633)
Epoch: ( 49) (  501/  633)
Epoch: ( 49) (  502/  633)
Epoch: ( 49) (  503/  633)
Epoch: ( 49) (  504/  633)
Epoch: ( 49) (  505/  633)
Epoch: ( 49) (  506/  633)
Epoch: ( 49) (  507/  633)
Epoch: ( 49) (  508/  633)
Epoch: ( 49) (  509/  633)
Epoch: ( 49) (  510/  633)
Epoch: ( 49) (  511/  633)
Epoch: ( 49) (  512/  633)
Epoch: ( 49) (  513/  633)
Epoch: ( 49) (  514/  633)
Epoch: ( 49) (  515/  633)
Epoch: ( 49) (  516/  633)
Epoch: ( 49) (  517/  633)
Epoch: ( 49) (  518/  633)
Epoch: ( 49) (  519/  633)
E

Epoch: ( 49) (  583/  633)
Epoch: ( 49) (  584/  633)
Epoch: ( 49) (  585/  633)
Epoch: ( 49) (  586/  633)
Epoch: ( 49) (  587/  633)
Epoch: ( 49) (  588/  633)
Epoch: ( 49) (  589/  633)
Epoch: ( 49) (  590/  633)
Epoch: ( 49) (  591/  633)
Epoch: ( 49) (  592/  633)
Epoch: ( 49) (  593/  633)
Epoch: ( 49) (  594/  633)
Epoch: ( 49) (  595/  633)
Epoch: ( 49) (  596/  633)
Epoch: ( 49) (  597/  633)
Epoch: ( 49) (  598/  633)
Epoch: ( 49) (  599/  633)
Epoch: ( 49) (  600/  633)
Epoch: ( 49) (  601/  633)
Epoch: ( 49) (  602/  633)
Epoch: ( 49) (  603/  633)
Epoch: ( 49) (  604/  633)
Epoch: ( 49) (  605/  633)
Epoch: ( 49) (  606/  633)
Epoch: ( 49) (  607/  633)
Epoch: ( 49) (  608/  633)
Epoch: ( 49) (  609/  633)
Epoch: ( 49) (  610/  633)
Epoch: ( 49) (  611/  633)
Epoch: ( 49) (  612/  633)
Epoch: ( 49) (  613/  633)
Epoch: ( 49) (  614/  633)
Epoch: ( 49) (  615/  633)
Epoch: ( 49) (  616/  633)
Epoch: ( 49) (  617/  633)
Epoch: ( 49) (  618/  633)
Epoch: ( 49) (  619/  633)
E